# Sentinel-2 Imagery Visualization

This notebook provides interactive tools for exploring Sentinel-2 satellite imagery of the mimosa bloom cycle in Mandelieu-la-Napoule.

**Features:**
1. Time-series viewer with animation showing bloom progression
2. Interactive band blending tool with preset composites, custom RGB builder, and single-band viewer

## Setup and Data Loading

In [ ]:
from pathlib import Path

import numpy as np
import plotly.graph_objects as go
from IPython.display import display
from ipywidgets import Dropdown, Output, RadioButtons, VBox

from mimosa import (
    COMPOSITE_PRESETS,
    INDEX_LAYERS,
    SENTINEL2_BANDS,
    calculate_moisture_index,
    calculate_ndsi,
    calculate_ndvi,
    calculate_ndwi,
    create_index_visualization,
    create_rgb_composite,
    discover_dates,
    get_band_label,
    load_all_bands,
    load_true_color,
    normalize_band,
)

In [ ]:
# Configuration
DATA_DIR = Path("../data")
dates = discover_dates(DATA_DIR)

print(f"Found {len(dates)} acquisition dates:")
for date in dates:
    print(f"  - {date.strftime('%Y-%m-%d')}")

In [ ]:
# Eager load true color images (manageable memory footprint)
print("Loading true color images...")
true_color_cache = {}
true_color_mask_cache = {}

for date in dates:
    img, mask = load_true_color(DATA_DIR, date)
    true_color_cache[date] = img
    true_color_mask_cache[date] = mask

print(f"Loaded {len(true_color_cache)} true color images")

# Lazy load bands (larger memory, load on demand)
band_cache = {}
band_mask_cache = {}


def get_bands_for_date(date):
    if date not in band_cache:
        print(f"Loading bands for {date.strftime('%Y-%m-%d')}...")
        bands, masks = load_all_bands(DATA_DIR, date)
        band_cache[date] = bands
        band_mask_cache[date] = masks
    return band_cache[date], band_mask_cache[date]

## 1. Time-Series Viewer

Explore how the landscape changes throughout the mimosa bloom cycle using the slider and play/pause controls.

In [ ]:
# Create animation frames for each date
frames = []
for i, date in enumerate(dates):
    img = true_color_cache[date]
    frames.append(
        go.Frame(
            data=[go.Image(z=img)],
            name=str(i),
            layout={
                "title": {"text": f"Mimosa Bloom - {date.strftime('%Y-%m-%d')}"}
            },
        )
    )

# Create figure with first frame
fig = go.Figure(
    data=[go.Image(z=true_color_cache[dates[0]])],
    frames=frames,
)

# Create slider steps
slider_steps = []
for i, date in enumerate(dates):
    slider_steps.append(
        {
            "args": [
                [str(i)],
                {
                    "frame": {"duration": 0, "redraw": True},
                    "mode": "immediate",
                    "transition": {"duration": 300},
                },
            ],
            "label": date.strftime("%Y-%m-%d"),
            "method": "animate",
        }
    )

# Add animation controls
fig.update_layout(
    title={
        "text": "Mimosa Bloom Cycle - Mandelieu-la-Napoule",
        "x": 0.5,
        "xanchor": "center",
    },
    width=1100,
    height=900,
    xaxis={"visible": False, "showticklabels": False},
    yaxis={"visible": False, "showticklabels": False},
    margin={"l": 10, "r": 10, "t": 80, "b": 10},
    updatemenus=[
        {
            "type": "buttons",
            "showactive": False,
            "x": 0.1,
            "y": 0,
            "xanchor": "left",
            "yanchor": "top",
            "buttons": [
                {
                    "label": "▶ Play",
                    "method": "animate",
                    "args": [
                        None,
                        {
                            "frame": {"duration": 1000, "redraw": True},
                            "fromcurrent": True,
                            "transition": {"duration": 300},
                        },
                    ],
                },
                {
                    "label": "⏸ Pause",
                    "method": "animate",
                    "args": [
                        [None],
                        {
                            "frame": {"duration": 0, "redraw": False},
                            "mode": "immediate",
                            "transition": {"duration": 0},
                        },
                    ],
                },
            ],
        }
    ],
    sliders=[
        {
            "active": 0,
            "steps": slider_steps,
            "x": 0.1,
            "len": 0.9,
            "xanchor": "left",
            "y": 0,
            "yanchor": "top",
            "pad": {"b": 10, "t": 50},
            "currentvalue": {
                "visible": True,
                "prefix": "Date: ",
                "xanchor": "right",
                "font": {"size": 16},
            },
            "transition": {"duration": 300},
        }
    ],
)

fig.show()

## 2. Interactive Band Blending Tool

Explore different spectral band combinations to identify mimosa blooms.

**Four modes:**
- **Preset Composite**: Copernicus Browser standard RGB composites (True Color, False Color, SWIR, etc.)
- **Index**: Calculated indices (NDVI, Moisture Index, NDWI, NDSI)
- **Custom RGB**: Build your own composite by assigning any band to R, G, B channels
- **Single Band**: View individual spectral bands in grayscale

In [ ]:
# Create widgets
date_dropdown = Dropdown(
    options=[(d.strftime("%Y-%m-%d"), d) for d in dates],
    value=dates[2],  # Default to peak bloom
    description="Date:",
)

mode_radio = RadioButtons(
    options=["Preset Composite", "Index", "Custom RGB", "Single Band"],
    value="Preset Composite",
    description="Mode:",
)

# Preset composite dropdown
preset_dropdown = Dropdown(
    options=list(COMPOSITE_PRESETS.keys()),
    value="True Color",
    description="Preset:",
)

# Index dropdown
index_dropdown = Dropdown(
    options=INDEX_LAYERS, value="NDVI", description="Index:"
)

# Custom RGB band selectors
band_options = [(get_band_label(b), b) for b in SENTINEL2_BANDS]
r_dropdown = Dropdown(options=band_options, value="B04", description="Red:")
g_dropdown = Dropdown(options=band_options, value="B03", description="Green:")
b_dropdown = Dropdown(options=band_options, value="B02", description="Blue:")

# Single band selector
single_band_dropdown = Dropdown(
    options=band_options, value="B08", description="Band:"
)

# Output area
output = Output()


def update_visualization(*args):
    """Master callback - regenerate visualization on any change."""
    with output:
        output.clear_output(wait=True)

        # Load bands and masks for selected date
        bands, masks = get_bands_for_date(date_dropdown.value)
        date_str = date_dropdown.value.strftime("%Y-%m-%d")

        # Generate image based on mode
        if mode_radio.value == "Preset Composite":
            preset = COMPOSITE_PRESETS[preset_dropdown.value]
            img = create_rgb_composite(
                bands, masks, preset["r"], preset["g"], preset["b"]
            )
            title = f"{preset_dropdown.value} - {date_str}"

        elif mode_radio.value == "Index":
            # Calculate the selected index
            if index_dropdown.value == "NDVI":
                index_data = calculate_ndvi(bands, masks)
            elif index_dropdown.value == "Moisture Index":
                index_data = calculate_moisture_index(bands, masks)
            elif index_dropdown.value == "NDWI":
                index_data = calculate_ndwi(bands, masks)
            elif index_dropdown.value == "NDSI":
                index_data = calculate_ndsi(bands, masks)

            # Create color visualization
            img = create_index_visualization(index_data)
            title = f"{index_dropdown.value} - {date_str}"

        elif mode_radio.value == "Custom RGB":
            img = create_rgb_composite(
                bands, masks, r_dropdown.value, g_dropdown.value, b_dropdown.value
            )
            rgb_label = (
                f"{r_dropdown.value}/{g_dropdown.value}/{b_dropdown.value}"
            )
            title = f"Custom RGB ({rgb_label}) - {date_str}"

        else:  # Single Band
            band_data = normalize_band(
                bands[single_band_dropdown.value],
                masks[single_band_dropdown.value],
            )
            img_gray = (band_data * 255).astype(np.uint8)
            img = np.stack([img_gray, img_gray, img_gray], axis=-1)
            band_label = get_band_label(single_band_dropdown.value)
            title = f"{band_label} - {date_str}"

        # Display with Plotly
        fig = go.Figure(go.Image(z=img))
        fig.update_layout(
            title={"text": title, "x": 0.5, "xanchor": "center"},
            width=1000,
            height=820,
            xaxis={"visible": False},
            yaxis={"visible": False},
            margin={"l": 10, "r": 10, "t": 60, "b": 10},
        )
        display(fig)


# Wire up callbacks
date_dropdown.observe(update_visualization, names="value")
mode_radio.observe(update_visualization, names="value")
preset_dropdown.observe(update_visualization, names="value")
index_dropdown.observe(update_visualization, names="value")
r_dropdown.observe(update_visualization, names="value")
g_dropdown.observe(update_visualization, names="value")
b_dropdown.observe(update_visualization, names="value")
single_band_dropdown.observe(update_visualization, names="value")

# Create UI layout
ui = VBox(
    [
        date_dropdown,
        mode_radio,
        preset_dropdown,
        index_dropdown,
        r_dropdown,
        g_dropdown,
        b_dropdown,
        single_band_dropdown,
        output,
    ]
)

display(ui)
update_visualization()  # Initial render

## Tips for Mimosa Detection

**All 10 Copernicus Browser Layers Implemented!**

### RGB Composites (Preset mode)
- **True Color (B4-B3-B2)**: Natural appearance. Good for orientation and identifying landmarks.
- **False Color (B8-B4-B3)**: Vegetation = red/magenta. Best for assessing plant density and health.
- **Highlight Optimized Natural Color (B4-B3-B2)**: Enhanced true color with better contrast.
- **False Color Urban (B12-B11-B4)**: Urban areas = white/purple, vegetation = green.
- **SWIR (B12-B8A-B4)**: Water content visualization. Darker green = denser vegetation.

### Calculated Indices (Index mode)
- **NDVI**: (B8-B4)/(B8+B4) - Vegetation health. Green = healthy, red = stressed/bare.
- **Moisture Index**: (B8A-B11)/(B8A+B11) - Water content. Green = moist, red = dry.
- **NDWI**: (B3-B8)/(B3+B8) - Water bodies. Green = water, red = land.
- **NDSI**: (B3-B11)/(B3+B11) - Snow detection (not relevant for mimosa, but included).

### For Mimosa Bloom Detection

**Best layers to compare pre-bloom vs peak bloom:**
1. **False Color (B8-B4-B3)**: Look for changes in red intensity - flowering may reduce NIR reflection
2. **NDVI**: Track vegetation vigor changes during bloom cycle
3. **Moisture Index**: Flowering plants may show different water content patterns
4. **SWIR (B12-B8A-B4)**: Detect structural changes from flowering

**Recommended workflow:**
1. Start with **False Color** on 2024-12-16 (pre-bloom) - note vegetation patterns
2. Switch to 2025-02-14 (peak bloom) - look for color shifts in mimosa areas
3. Try **NDVI** to quantify vegetation changes (lower values may indicate flowering)
4. Use **Custom RGB** mode to experiment: try B11-B08-B04 for moisture+health visualization